# Download the necessary packages
Recommend using Linux or WSL for Windows.
A good Nvidia GPU (rtx-30xx/40xx) is required for reasonable speed.

In [ ]:
!nvidia-smi

The following install the required package for cuda 12.1 in Linux, for the newer RTX 30xx GPUs or higher 

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install --upgrade --force-reinstall --no-cache-dir torch==2.2.0 triton \
  --index-url https://download.pytorch.org/whl/cu121
!python -m pip install "unsloth[cu121-ampere-torch220] @ git+https://github.com/unslothai/unsloth.git"

If installed without failure, starts by downloading the model we are gonna use.

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    dtype=None,        # None for auto detect
    load_in_4bit=True,
) # about 6GB in size

In [ ]:
import torch

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Rate the following restaurant in the category of ??? from 1 to 5 where 1 means ##### and 5 means ######."},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors="pt"
).to(model.device)

rating_ids = []
for score in [1,2,3,4,5]:
    rating_ids.append(tokenizer.encode(score))

with torch.no_grad():
    output = model(input_ids=input_ids)[-1]